In [2]:
import cv2
import numpy as np

image = cv2.imread('images/signatures/drsc/forgery_9.jpg',0)
#cv2.imshow('Original', image)
#cv2.waitKey()

image = scale(image)
#cv2.imshow('Scaled', image)
#cv2.waitKey()

ret, image = cv2.threshold(image, 127, 255, cv2.THRESH_BINARY)
#cv2.imshow('Binarized', image)
#cv2.waitKey()

kernel = np.ones((1,1), np.uint8)
image = cv2.erode(image, kernel, iterations = 1)

#image = cv2.GaussianBlur(image, (7,7), 0)
#cv2.imshow("Gaussian Blurring", image)
#cv2.waitKey(0)

_,contours,hierarchy = cv2.findContours(255 - image, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_NONE)

max_area_contour = cv2.contourArea(contours[0])
max_index = 0
for index, contour in enumerate(contours):
    index_area_contour = cv2.contourArea(contour)
    if index_area_contour > max_area_contour:
        max_area_contour = index_area_contour
        max_index = index

cnt = contours[max_index]    
image = cv2.cvtColor(image, cv2.COLOR_GRAY2BGR)
#cv2.drawContours(image, [cnt], -1, (0,0,255), 2)
x,y,w,h = cv2.boundingRect(cnt)
#cv2.rectangle(image,(x,y),(x+w,y+h),(0,255,0),2)

cv2.imshow("Contours", image)

stencil = np.ones(image.shape).astype(image.dtype) * 255
cv2.fillPoly(stencil, [cnt], (0,0,0))
cv2.imshow("Stencil", stencil)

image = cv2.bitwise_or(image, stencil)

cv2.imshow("Limpio", image)


image = image[y:y+h,x:x+w]
cv2.imshow("Cropped", image)

cv2.waitKey(0)


cv2.destroyAllWindows()

In [31]:
cv2.imshow("Limpio", stencil)
cv2.waitKey(0)
cv2.destroyAllWindows()

In [1]:
def scale(image, height=500):
    # Obtenemos las filas y columnas
    original_h, original_w = np.float32(image.shape[:2])
    # Calculamos el ancho que tendra con respecto al alto(100)
    width = int( height * ( original_w / original_h ) )
    # Escalamos la imagen
    image_scaled = cv2.resize(image, (width, height), interpolation = cv2.INTER_AREA)
    # Retornamos la imagen escalada
    return image_scaled